<a href="https://colab.research.google.com/github/PLEX-GR00T/Maze_solving_MDP/blob/main/Q_learning_(Taxi_v3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMPE249_HW3_RL_SmartCab

In this problem, you are going to implement the basic Q-learning algorithm to teach a Smartcab to pick up the passenger at one location and drop them off in another. The goals include:
1. Drop off the passenger to the right location. 
2. Find the minimum path.
3. Avoid obstacles and follow traffice rules.

Fortunately, OpenAI Gym (https://gym.openai.com/) has a simualtion environment already built for this problem. 

You need to install "gym" first if you have not done so already using

!pip install cmake 'gym[atari]' scipy


Load the game environment and render what it looks like. 
The filled square represents the taxi, which is yellow without a passenger and green with a passenger.
The pipe ("|") represents a wall which the taxi cannot cross.
R, G, Y, B are the possible pickup and destination locations. The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.

In [1]:
import gym
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Here's the restructured problem statement (from Gym docs):

"There are 4 locations (labeled by different letters), and the job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions."

The action space include six actions:
  0 = south
  1 = north
  2 = east
  3 = west
  4 = pickup
  5 = dropoff





In [2]:
# reset the environment to a new, random state
env.reset() 
env.render()
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


When the Taxi environment is created, there is an initial Reward table that's also created, called `P`. We can think of it like a matrix that has the number of states as rows and number of actions as columns, i.e. a states × actions matrix.

Since every state is in this matrix, we can see the default reward values assigned to one of the state 328:

In [3]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

This dictionary has the structure {action: [(probability, nextstate, reward, done)]}.

A few things to note:

1. The 0-5 corresponds to the actions (south, north, east, west, pickup, dropoff) the taxi can perform at our current state in the illustration.
2. In this env, "probability" is always 1.0.
3. The "nextstate" is the state we would be in if we take the action at this index of the dict
4. All the movement actions have a -1 reward and the pickup/dropoff actions have -10 reward in this particular state. If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action (5)
5. "done" is used to tell us when we have successfully dropped off a passenger in the right location. Each successfull dropoff is the end of an episode

Note that if our agent chose to explore action two (2) in this state it would be going East into a wall. The source code has made it impossible to actually move the taxi across a wall, so if the taxi chooses that action, it will just keep accruing -1 penalties, which affects the long-term reward.

Now, let's use Q-learning to solve this problem.

First, we will initialize the Q-table to a 500 * 6 matrix of zeros.

In [14]:
import numpy
q_table = np.zeros([env.observation_space.n, env.action_space.n])

TODO: implement the Q-learning algorithm to find the best strategy

In [15]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    #TODO: implement the Q-learning algorithm
    while not done:
      if random.uniform(0,1) < epsilon:
        action = env.action_space.sample()
      else:
        action = numpy.argmax(q_table[state])

      nxtState, reward, done, info = new = env.step(action)

      oldValue =  q_table[state,action]
      nxtMax = numpy.max(q_table[nxtState])


      newValue = (1-alpha) * oldValue + alpha * (reward + gamma * nxtMax)
      q_table[state, action] = newValue

      if reward == -10:
        penalties += 1
      
      state = nxtState

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 1min 7s, sys: 14.2 s, total: 1min 21s
Wall time: 1min 9s


let's see what the Q-values are at state 328:

In [16]:
q_table[328]

array([ -2.4131659 ,  -2.27325184,  -2.40018244,  -2.3604151 ,
       -10.79628063, -10.67814863])

In [17]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.93
Average penalties per episode: 0.0
